In [2]:
from models.vgg16 import VGG16
from models.mlp import MLP
from models.encoder import TransformerEncoder
from utils.config_manager import ConfigManager
from pathlib import Path
from dataset.dataset import CropInfoDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import timm
config = ConfigManager()
VGG_WEIGHTS_PATH = str('model_weights/vgg16.bin')
class WaterPredictModel(nn.Module):
    def __init__(self):
        super(WaterPredictModel, self).__init__()
        vgg16_conf = config.get('vgg16')
        self.vgg16 = VGG16(VGG_WEIGHTS_PATH, num_classes=vgg16_conf['output_dim'])
        self.data_config = timm.data.resolve_model_data_config(self.vgg16.model)
        
        encoder_conf = config.get('encoder')
        input_dim, d_model, nhead, nhid, nlayers, dropout = \
            encoder_conf['input_dim'], encoder_conf['d_model'], encoder_conf['nhead'],\
                 encoder_conf['nhid'], encoder_conf['nlayers'], encoder_conf['dropout']
        self.transformer = TransformerEncoder(
            input_dim=input_dim, d_model=d_model, nhead=nhead, nhid=nhid, nlayers=nlayers, dropout=dropout
        )
        mlp_conf = config.get('mlp')
        input_dim, hidden_dim, output_dim = \
            mlp_conf['input_dim'], mlp_conf['hidden_dim'], mlp_conf['output_dim']
        self.mlp = MLP(input_size=input_dim, hidden_size=hidden_dim, num_classes=output_dim)
    def get_transformer(self, is_training=False):
        return timm.data.create_transform(**self.data_config, is_training=is_training)
    def forward(self, image, t_T_data):
        image_embd = self.vgg16(image)
        t_T_embd = self.transformer(t_T_data)
        # print(image_embd.shape)
        # print(t_T_embd.shape)
        embd = torch.cat((image_embd, t_T_embd), dim=1)
        return self.mlp(embd)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = WaterPredictModel().to(device)
dataset = CropInfoDataset('dataset/preprocess/rgb_images_list.json', 'dataset/preprocess/t_T_mo_data.json', transform=model.get_transformer(is_training=False))
test_dataloader = DataLoader(dataset, batch_size=2, shuffle=False, num_workers=1)
# 示例代码
model.eval()  # 将模型设置为评估模式
    

e:\onedrive\桌面\毕业设计\算法\venv\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


WaterPredictModel(
  (vgg16): VGG16(
    (model): VGG(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace=True)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace=True)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace=True)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace=True)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1

In [5]:
with torch.no_grad():  # 禁用梯度计算
    for images, t_T_moisture, labels in test_dataloader:  # 假设test_dataloader是测试数据
        images, t_T_moisture = images.to(device), t_T_moisture.to(device)
        outputs = model(images, t_T_moisture)
        print(outputs)
        break
        # 进行预测处理，例如取最大概率的标签等

tensor([[-0.0769],
        [-0.0820]])
